In [1]:
import xarray as xr
import numpy as np
import sys, os, glob
import time
from scipy.stats import weibull_min
import dask.distributed as dd

root_dir = '/media/harish/SSD_4TB/EU_SCORES_project'
scripts_dir = f'{root_dir}/scripts'
sys.path.append(scripts_dir)

from data_processing.libraries import mean_statistics, std_statistics, wind_power_density, weibull , weibull_statistics

In [8]:
root_dir = '/media/harish/SSD_4TB/EU_SCORES_project'
run_dir = f'{root_dir}/CERRA'
region = 'Portugal'
file_names=['ws10', 'ws100', 'wpd_100', 't2m', 'swdown']
variable = 'ws'
chunks={"time": 3600,"x": -1,"y": -1}
for file_name in file_names:
    ds = xr.open_dataset(f'{run_dir}/{region}/variablewise_files/{file_name}.nc',chunks=chunks) 
    # check if 'variable' variable exist in ds
    if 'variable' in ds:
        ds = ds.drop('variable')
    data = ds.isel(time=slice(None,-1),variable=0)
    print(data)

<xarray.Dataset>
Dimensions:            (time: 90336, y: 41, x: 52)
Coordinates:
  * time               (time) datetime64[ns] 1990-01-01 ... 2020-12-31T21:00:00
    step               timedelta64[ns] ...
    heightAboveGround  float64 ...
    latitude           (y, x) float64 dask.array<chunksize=(41, 52), meta=np.ndarray>
    longitude          (y, x) float64 dask.array<chunksize=(41, 52), meta=np.ndarray>
    valid_time         (time) datetime64[ns] dask.array<chunksize=(3600,), meta=np.ndarray>
Dimensions without coordinates: y, x
Data variables:
    ws                 (time, y, x) float32 dask.array<chunksize=(3600, 41, 52), meta=np.ndarray>
<xarray.Dataset>
Dimensions:            (time: 90584, y: 41, x: 52)
Coordinates:
  * time               (time) datetime64[ns] 1990-01-01 ... 2020-12-31T21:00:00
    step               timedelta64[ns] ...
    heightAboveGround  float64 ...
    latitude           (y, x) float64 dask.array<chunksize=(41, 52), meta=np.ndarray>
    longitude        

In [4]:
# === mean ===#
print('Calculating mean statistics')
mean_statistics(data,time_coord='time').to_netcdf(f'{target_dir}/mean.nc')

# === std ===#
print('Calculating std statistics')
std_statistics(data,time_coord='time').to_netcdf(f'{target_dir}/std.nc')

# === CoV ===#
print('Calculating CoV statistics')
std = xr.open_dataset(f'{target_dir}/std.nc')
mean = xr.open_dataset(f'{target_dir}/mean.nc')
cov = (std/mean)
cov.to_netcdf(f'{target_dir}/cov.nc')

# === 5th quantile ===#
print('Calculating 5th quantile statistics')
quantile_statistics(data,0.05,time_coord='time').to_netcdf(f'{target_dir}/quantile_5.nc')

# === 95th quantile ===#
print('Calculating 95th quantile statistics')
quantile_statistics(data,0.95,time_coord='time').to_netcdf(f'{target_dir}/quantile_95.nc')

# === 99th quantile ===#
print('Calculating 99th quantile statistics')
quantile_statistics(data,0.99,time_coord='time').to_netcdf(f'{target_dir}/quantile_99.nc')

Hourly statistics calculated in 1.6381034851074219 seconds
Monthly statistics calculated in 0.7320060729980469 seconds
Yearly statistics calculated in 1.3601555824279785 seconds
Overall statistics calculated in 0.004815816879272461 seconds


<xarray.Dataset>
Dimensions:            (y: 41, x: 52, hour: 8, month: 12, year: 31)
Coordinates:
    step               timedelta64[ns] 00:00:00
    heightAboveGround  float64 10.0
    latitude           (y, x) float64 40.25 40.26 40.28 ... 42.7 42.71 42.72
    longitude          (y, x) float64 349.7 349.7 349.8 ... 352.2 352.3 352.3
  * hour               (hour) int64 0 3 6 9 12 15 18 21
  * month              (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * year               (year) int64 1990 1991 1992 1993 ... 2017 2018 2019 2020
Dimensions without coordinates: y, x
Data variables:
    hourly_values      (hour, y, x) float32 7.349 7.33 7.314 ... 2.492 2.417
    monthly_values     (month, y, x) float32 7.764 7.747 7.736 ... 2.341 2.272
    yearly_values      (year, y, x) float32 7.068 7.035 7.012 ... 2.464 2.44
    overall_values     (y, x) float32 7.28 7.257 7.242 ... 2.632 2.385 2.32